# FASTTEXT

# (1) Скачиваем FASTTEXT

In [0]:
!wget 'http://vectors.nlpl.eu/repository/11/181.zip'

--2019-10-25 01:40:11--  http://vectors.nlpl.eu/repository/11/181.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2622716217 (2.4G) [application/zip]
Saving to: ‘181.zip.2’

181.zip.2            14%[=>                  ] 356.19M  16.9MB/s    eta 2m 59s 

KeyboardInterrupt: ignored

In [0]:
!unzip '181.zip' -d 'fasttext'

Archive:  181.zip
  inflating: fasttext/meta.json      
  inflating: fasttext/model.model    
  inflating: fasttext/model.model.vectors_ngrams.npy  
  inflating: fasttext/model.model.vectors.npy  
  inflating: fasttext/model.model.vectors_vocab.npy  
  inflating: fasttext/README         


In [0]:
from gensim.models.keyedvectors import KeyedVectors

fast_model = 'fasttext/model.model'

In [0]:
fasttext_model = KeyedVectors.load(fast_model)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# (2) Скачиваем корпус документов

In [0]:
import pandas as pd

In [0]:
!wget 'https://www.dropbox.com/s/jaa5y82qzul6byn/quora_question_pairs_rus.csv'

--2019-10-24 23:04:15--  https://www.dropbox.com/s/jaa5y82qzul6byn/quora_question_pairs_rus.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/jaa5y82qzul6byn/quora_question_pairs_rus.csv [following]
--2019-10-24 23:04:15--  https://www.dropbox.com/s/raw/jaa5y82qzul6byn/quora_question_pairs_rus.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucba04202d73e66d5a4e8ade8d70.dl.dropboxusercontent.com/cd/0/inline/ArGze7ayNVdqXUP9S_JrTEjh5S_Q2eCvnIM40Yiu-axIb3ugo2zxFwqHHH_YHb1aqZjtpHWC8Cj9CHtaEtH_SpMzcTvPVTgU1EHGhyYkwXY5OQ7oHeubnTj79_MiUjVttpk/file# [following]
--2019-10-24 23:04:16--  https://ucba04202d73e66d5a4e8ade8d70.dl.dropboxusercontent.com/cd/0/inline/ArGze7ayNVdqXUP9S_JrTEjh5S_Q2eCvnIM40Yiu-axIb3ugo2zxFwqHHH_YHb1aqZ

In [0]:
data = pd.read_csv('quora_question_pairs_rus.csv', index_col='Unnamed: 0')

In [0]:
data=data[:1000] # взяла первые 1000 запросов, чтобы было удобно сравнить с точностью поиска по bm25

# (3) Скачиваем словарь модели 

In [0]:
import pickle
import numpy as np

In [0]:
lemmas = fasttext_model.vocab

# создаем пустую матрицу размерности длина словаря на 300
lemmas_vectors = np.zeros((len(lemmas), fasttext_model.vector_size))

# добавляем векторы для каждой леммы
for idx, lemma in enumerate(lemmas):
    lemmas_vectors[idx] = fasttext_model[lemma]
    
mean_vector = lemmas_vectors.mean(axis=0)

# сохраняем в pickle файл
with open('lemmas_vocab.pickle', 'wb') as f:
  pickle.dump(lemmas_vectors, f)

# (4) Создаем матрицу по документам

In [0]:
! pip install pymorphy2

     |████████████████████████████████| 51kB 2.4MB/s 
     |████████████████████████████████| 7.1MB 5.7MB/s 


In [0]:
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import WordPunctTokenizer
import re
import time
import os
from sklearn.feature_extraction.text import CountVectorizer
from math import log
import nltk
morph = MorphAnalyzer()

In [0]:
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def preprocessing(text):
    
    text = re.sub(r'[A-Za-z0-9<>«»\.!\(\)?,;:\-\"\ufeff]', r'', str(text))
    text = WordPunctTokenizer().tokenize(text)
    preproc_text = ''
    tokens = [morph.parse(token)[0] for token in text]
    normal_forms = [token.normal_form for token in tokens if token not in russian_stopwords and token.normal_form != 'это']
    preproc_text = ' '.join(normal_forms)
       
    return preproc_text

In [0]:
# препроцессим корпус
%%time
data['question1_lemm'] = data['question1'].apply(lambda x: preprocessing(x))

CPU times: user 17.8 s, sys: 0 ns, total: 17.8 s
Wall time: 17.9 s


In [0]:
def sent_vectorizer(sent, model, lemmas_vectors): # превращает каждое слово в вектор
   
    if type(sent) != str:
        
        sent_vector = np.zeros((model.vector_size,))
        return sent_vector
    
    sent = sent.split()
    lemmas_vectors = np.zeros((len(sent), model.vector_size))
    vec = np.zeros((model.vector_size,))
    
    for idx, lemma in enumerate(sent):
        if lemma in model.vocab:
            lemmas_vectors[idx] = model[lemma]
    
    sent_vector = lemmas_vectors.sum(axis=0) # суммируем вектор для каждого документа
    return sent_vector

In [0]:
with open('lemmas_vocab.pickle', 'rb') as f:
  lemmas = pickle.load(f)

In [0]:
# записываем в матрицу просуммированные векторы для каждого документа
%%time
data['q1_vector'] = data['question1_lemm'].apply(lambda x: sent_vectorizer(x, fasttext_model, lemmas))

CPU times: user 454 ms, sys: 29.9 ms, total: 484 ms
Wall time: 488 ms


In [0]:
with open('docs_vectors.pickle', 'wb') as f:
  pickle.dump(data, f)

# (5) Поиск на FastText

In [0]:
# получаем запрос
query = input('Введите Ваш запрос: ')

Введите Ваш запрос: rcytvubinjok


In [0]:
!pip install pandas==0.25.2

     |████████████████████████████████| 10.4MB 4.9MB/s 
  Found existing installation: pandas 0.24.1
    Uninstalling pandas-0.24.1:
      Successfully uninstalled pandas-0.24.1


In [0]:
# лемматизируем запрос
query_lemm = preprocessing(query)

In [0]:
# получаем вектор запроса
query_vec = sent_vectorizer(query_lemm, fasttext_model, lemmas)

In [0]:
# считаем косинусную близость по запросу для каждого документа
from sklearn.metrics.pairwise import cosine_similarity

def cos_similarity(docs_vectors, query_vec):
  cos_sims = {} # словарь косинусных близостей запроса к каждому доку
  
  for i, vector in enumerate(docs_vectors['q1_vector']):
    cos_sim = cosine_similarity(vector.reshape(1, -1), query_vec.reshape(1, -1))
    cos_sims[i] = cos_sim[0][0]
  
  return cos_sims

In [0]:
with open('docs_vectors.pickle', 'rb') as f:
  docs_vectors = pickle.load(f)

cos_sims = cos_similarity(docs_vectors, query_vec)

# сортируем по косинусной близости
result = sorted(cos_sims.items(), key=lambda x: (x[1],x[0]), reverse=True)

# печатаем топ 10 документов
if result[:1][0][1] == float(0):
  print('По Вашему запросу ' + query + ' ничего не найдено')

else:
  for r in result[:10]:
    print('РЕЛЕВАНТНЫЙ ДОКУМЕНТ: '+ docs_vectors.at[r[0], 'question1'] +'; КОСИНУСНАЯ БЛИЗОСТЬ РАВНА', r[1])

По Вашему запросу rcytvubinjok ничего не найдено


# (6) Качество поиска на FastText

In [0]:
def accuracy(top_results, data):
  
  ACCURACY = 0
  
  for res in top_results:
    #print(res[1], data.at[res[0], 'is_duplicate'])
    
    if res[1] > 0 and data.at[res[0], 'is_duplicate'] == 1:
      ACCURACY = 1
    
    else:
      ACCURACY = ACCURACY
  
  return ACCURACY

In [0]:
with open('lemmas_vocab.pickle', 'rb') as f:
  lemmas = pickle.load(f)
  
with open('docs_vectors.pickle', 'rb') as f:
  data = pickle.load(f)

data['question2_lemm'] = data['question2'].apply(lambda x: preprocessing(x))
data['q2_vector'] = data['question2_lemm'].apply(lambda x: sent_vectorizer(x, fasttext_model, lemmas))

In [0]:
accuracies = 0
for i, query in enumerate(data['q2_vector']):

  cos_sims = cos_similarity(docs_vectors, query)
  
  result = sorted(cos_sims.items(), key=lambda x: (x[1],x[0]), reverse=True)
  accuracies += accuracy(result[:5], data)
  

print('Точность поиска на fasttext равна ', accuracies/len(data))

Точность поиска на fasttext равна  0.819
